<a href="https://colab.research.google.com/github/tufts-mathmodeling/HW/blob/master/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import cvxpy as cp

# Math Modeling Homework 2 (Spring 2020)

# Part I: Root finders and finite differences

### Newton's method
As described in class, Newton's method gives an approximation $x_k$ to $f(x)$ using the tangent line at the previous iterate, $x_{k-1}$. Convergence is guaranteed if the initial guess, $x_0$, is close enough to the root, and if $f$ is continuously differentiable near the root. Both $f(x)$ and $f'(x)$ are required for implementation.

In [0]:
def newton(f, df, x0, max_steps=100, tol=1e-10):
    """Computes iterates of Newton's method for solving f(x) = 0.
    
    :param f: The function to find a root for.
    :param df: The derivative of the function.
    :param x0: The initial guess.
    :return: The final iterate and the total number of iterations needed.
    """
    x = x0
    for iteration in range(max_steps):
        if abs(f(x)) <= tol:
            return x, iteration
        x -= f(x) / df(x)
    return x, max_steps

### Bisection method
The bisection algorithm assumes that you know two points, $x_\ell$ and $x_r$, for which $f(x_\ell)$ and $f(x_r)$ have opposite signs.  For continuous $f$, this implies there is a zero between $x_\ell$ and $x_r$. **(Why?)** To find it, the algorithm iteratively divides the interval from $x_\ell$ to $x_r$ into two parts by examining the sign of $f((x_\ell+x_r)/2)$ and discarding the endpoint of the same sign.  No assumption on $f$ is made aside from continuity, and convergence is always guaranteed.  Only $f(x)$ is needed for implementation.

In [0]:
def bisection(f, x_left, x_right, max_steps=100, tol=1e-10):
    """Computes iterates of the bisection method for solving f(x) = 0.
    
    The signs of f(x_left) and f(x_right) must differ.
    
    :param f: The function to find a root for.
    :param x_left: A guess to the left of the root.
    :param x_right: A guess to the right of the root.
    """
    f_left = f(x_left)
    f_right = f(x_right)
    assert (f_left * f_right).real <= 0 # Signs of initial guesses must differ.
    
    x_middle = (x_left + x_right) / 2
    f_middle = f(x_middle)
    for iteration in range(max_steps):
        if abs(f_middle) <= tol:
            return x_middle, iteration
        # Narrow the search.
        if (f_left * f_middle).real > 0:
            # The left guess and the middle guess have the same sign.
            # Replace the left guess with the middle guess.
            x_left = x_middle
            f_left = f_middle
        else:
            # The right guess and the middle guess have the same sign.
            # Replace the right guess with the middle guess.
            x_right = x_middle
            f_right = f_middle
        # Compute the new middle point.
        x_middle = (x_left + x_right) / 2
        f_middle = f(x_middle)
    return x_middle, max_steps

### Secant method
As briefly mentioned in class, the secant method gives an approximation $x_k$ to $f(x)$ by the secant line between two previous points, $x_{k-1}$ and $x_{k-2}$.  Convergence is guaranteed if the initial guesses, $x_0$ and $x_1$ are close enough to the root, and if $f$ is twice continuously differentiable near the root with nonzero first derivative at the root.  Despite these weaker theoretical guarantees of convergence, an advantage is that only $f(x)$ is needed for implementation.

In [0]:
def secant(f, x0, x1, max_steps=100, tol=1e-10):
    """Computes iterates of the secant method for solving f(x) = 0.
    
    :param f: The function to find a root for.
    :param x0: A first initial guess.
    :param x1: A second initial guess.
    :return: The final iterate and the total number of iterations needed.
    """
    x_old = x0
    x_new = x1
    f_old = f(x_old)
    f_new = f(x_new)
    for iteration in range(max_steps):
        if abs(f(x_new)) <= tol:
            return x_new, iteration
        x_next = x_new - (f_new * (x_new - x_old) / (f_new - f_old))
        x_old = x_new
        f_old = f_new
        x_new = x_next
        f_new = f(x_next)
    return x_new, max_steps

### Halley's method
Halley's method (same Edmond Halley of Halley's comet) approximates $f(x)$ by a ratio of linear functions determined by the previous iterate.  The update formula is given by

\begin{equation}
x_{k} = x_{k-1} -
\frac{2f(x_{k-1})f'(x_{k-1})}{2\left(f'(x_{k-1})\right)^2 - f(x_{k-1})f''(x_{k-1})}.
\end{equation}

Convergence is guaranteed if the initial guess, $x_0$, is close enough to the root, and if $f$ is three times continuously differentiable near the root.  All of $f(x)$, $f'(x)$, and $f''(x)$ are required for implementation.

## Python concept we're using: passing functions to functions

All four root-finding functions take functions as arguments. This is a common pattern used in optimization libraries [such as scipy.optimize](https://docs.scipy.org/doc/scipy/reference/optimize.html). In the example below, we define Python functions that correspond to the polynomial $f(x) = x^2 - x - 2$ and its first and second derivative. We then pass these functions to the root-finding functions and compare the results.

In [0]:
def quadratic_f(x):
    return x**2 - x - 2

def quadratic_df(x):
    return (2 * x) - 1 # first derivative of f(x) -- manually computed

def quadratic_ddf(x):
    return 2 # second derivative of f(x) -- manually computed

root_bisect, iter_bisect = bisection(quadratic_f, -5, 0)
root_newton, iter_newton = newton(quadratic_f, quadratic_df, 0)
root_secant, iter_secant = secant(quadratic_f, -5, 0)
root_halley, iter_halley = halley(quadratic_f, quadratic_df, quadratic_ddf, 0)
print(f'The bisection method took {iter_bisect} iterations to find the root x={root_bisect}.')
print(f'The Newton method took {iter_newton} iterations to find the root x={root_newton}.')
print(f'The secant method took {iter_secant} iterations to find the root x={root_secant}.')
print(f'The Halley method took {iter_halley} iterations to find the root x={root_halley}.')

The bisection method took 35 iterations to find the root x=-0.9999999999854481.
The Newton method took 6 iterations to find the root x=-1.0.
The secant method took 8 iterations to find the root x=-1.0000000000000013.
The Halley method took 4 iterations to find the root x=-1.0.


Sometimes, it is cumbersome to define functions in the way illustrated above. Rather than using the `def` keyword, we can use [one-line _lambda expressions_](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) to create anonymous functions—that is, functions without an explicit name. Here's how we might rewrite the root-finding problem above as a series of lambda expressions:

In [0]:
root_halley, iter_halley = halley(lambda x: x**2 - x - 2,
                                  lambda x: (2 * x) - 1,
                                  lambda x: 2,
                                  0)
print(f'The Halley method took {root_halley} iterations to find the root x={iter_halley}.')

The Halley method took 11 iterations to find the root x=0.9999943549689424.



**Now your actual homework problems...**

### Problem 1: Microprocessor division with Newton's method (8 points)
One of the uses of Newton's method is in implementing division on microprocessors, where only addition and multiplication are available as primitive operations.  To compute $x=\frac{a}{b}$, first the root of $f(x) = \frac{1}{x} - b$ is found using Newton's method, then the fraction is computed with one last multiplication by $a$.  

**(i)** Find the Newton iteration needed to solve $f(x)=0$ and explain why it is well-suited to this purpose.  *(Note: We are trying to approximate division, so we shouldn't actually use it...)*

**(ii)** Explain why Halley's method is not well-suited to this purpose.

**(iii)** Apply Newton's method to computing $1/b$, where $b$ is (a) the last 3 digits of your student ID number; and (b) the area code of your phone number. For these experiments, report the iterates until the approximation is consistent with the last iterate to 10 digits (you will need to modify the given code in order to do so). For instance, `newton_division(1, 3, 0.01)` should print something like:
```
Iteration 0: 1/b = 0.01
Iteration 1: 1/b = 0.0197
Iteration 2: 1/b = 0.038235729999999996
Iteration 3: 1/b = 0.07208554685410129
Iteration 4: 1/b = 0.1285821155124381
Iteration 5: 1/b = 0.20756414973591425
Iteration 6: 1/b = 0.2858796707050494
Iteration 7: 1/b = 0.3265777830428163
Iteration 8: 1/b = 0.3331964209541502
Iteration 9: 1/b = 0.33333327709833466
Iteration 10: 1/b = 0.3333333333333238
Iteration 11: 1/b = 0.3333333333333333
```


### Problem 2: Square roots with Newton's method (4 points)

Another use for Newton's method and other rootfinding approaches is for computing square roots, by solving for zeros of $f(x) = x^2-a$.  Apply all of the above methods to computing $\sqrt{a}$ where $a$ is (a) the last 3 digits of your student number and (b) the area code of your phone number. Explain how you choose good initial guesses, and report the number of iterations needed for convergence.

In [0]:
def newton_sqrt(a, x0):
    """Finds the square root of a number using Newton's method.
    
    Your code should pass a function and its derivative to the
    newton() function and return the square root of `a` and the
    number of iterates required.
    
    :param a: The number to find a square root of.
    :param x0: An initial guess of the square root.
    """
    # TODO: Your code here!

In [0]:
# Unit tests for newton_division().
# Use these to make sure your implementation is behaving properly.
sqrt_tol = 1e-5
for a in (0, 0.001, 0.01, 0.1, 1, 10, 100, 1000):
    sqrt_a, _ = newton_sqrt(a, 1)
    assert abs(sqrt_a - np.sqrt(a)) <= sqrt_tol

In [0]:
# TODO: Your code here!
# Use newton_sqrt() to compute the square root of the last 3 digits
# of your student number and the area code of your phone number.
# Print the square roots and the number of iterations necessary to find them.

### Problem 3: Comparison of methods (4 points)
In this problem, we'll look at performance of various methods for functions, with and without the conditions needed for the theoretical guarantee of convergence.  Consider the 3 functions
 
\begin{align*}
f_1(x) & = x^3 \\
f_2(x) & = x^3 + 2x \\
f_3(x) & = x^3 + 2x^{5/4}
\end{align*}

Report on the iteration counts for each algorithm for each function to find the root at $x=0$ for consistent initial guesses (be sure to avoid cases where the algorithms get "lucky" and find the solution $x=0$ at their first step).  You will be graded on your discussion of what you observe.

In [0]:
labels = ['x³', 'x³ + 2x', 'x³ + 2x⁵⁄⁴']

functions = [
    lambda x: x**3,
    lambda x: x**3 + (2 * x),
    lambda x: x**3 + (2 * x**1.25)
]

df_funcs = [
    # TODO: Your code here!
    # Write lambda expressions for the first derivatives
    # of the three functions above.
]

ddf_funcs = [
    # TODO: Your code here!
    # Write lambda expressions for the second derivatives
    # of the three functions above.
]

Now we just report the comparisons.  Experiment with the parameters here and make sure you understand the role they play.

In [0]:
for label, f, df, ddf in zip(labels, functions, df_funcs, ddf_funcs):
    print('---', label, '---')
    root_bisect, iter_bisect = bisection(f, -2, 5)
    root_newton, iter_newton = newton(f, df, 5)
    root_secant, iter_secant = secant(f, -2, 5)
    root_halley, iter_halley = halley(f, df, ddf, 5)
    print(f'Bisect:\t{iter_bisect} iterations, x={root_bisect}')
    print(f'Newton:\t{iter_newton} iterations, x={root_newton}')
    print(f'Secant:\t{iter_secant} iterations, x={root_secant}')
    print(f'Halley:\t{iter_halley} iterations, x={root_halley}')
    print()

--- x³ ---
Bisect:	12 iterations, x=0.0003662109375
Newton:	23 iterations, x=0.0004455239404766183
Secant:	31 iterations, x=-0.0003789623402101401
Halley:	14 iterations, x=0.00030517578125

--- x³ + 2x ---
Bisect:	34 iterations, x=-2.9103830456733704e-11
Newton:	8 iterations, x=0.0
Secant:	8 iterations, x=-1.938464878150942e-11
Halley:	5 iterations, x=3.324604317973129e-20

--- x³ + 2x⁵⁄⁴ ---
Bisect:	28 iterations, x=-1.862645149230957e-09
Newton:	16 iterations, x=4.459017656768588e-09
Secant:	21 iterations, x=(-9.414029569228546e-10+2.1845973164009137e-09j)
Halley:	11 iterations, x=3.0392007280703785e-09



## Part 2: Linear programming
In this part of the HW, we will use the [CVXPY](https://www.cvxpy.org/index.html) convex optimization package to solve linear programs. The basic syntax of CVXPY maps nicely to informal ways of writing linear programs. For instance, this linear program in two variables


\begin{align*}
\text{minimize  }2x + 3y & \\
\text{subject to } x + y &\geq 1 \\
y-x &\leq 3 \\
2x+y &\leq 9 \\
x-y &\leq 3 \\
x & \geq 0 \\
y & \geq 0
\end{align*}

can be expressed by just listing these inequalities as:

In [0]:
x = cp.Variable()
y = cp.Variable()
objective = cp.Minimize((2 * x) + (3 * y))
constraints = [
    x + y >= 1,
    y - x <= 3,
    (2 * x) + y <= 9,
    x - y <= 3,
    x >= 0,
    y >= 0
]
prob = cp.Problem(objective, constraints=constraints)

We'll call this the **list form** of a linear program.  We can use the `solve()` method to solve the program. A solver used internally by CVXPY [(learn more)](https://www.cvxpy.org/tutorial/advanced/index.html#choosing-a-solver) finds the optimal values for $x$ and $y$ with respect to the objective and the six constraints. The `solve()` method returns the minimum value of the objective function as a convenience.

We'll use [floating point](https://en.wikipedia.org/wiki/Floating-point_arithmetic) numbers and specify a number of decimal places we want.  For instance, "{0:.4f}".format(0.1234567890) would return 0.1235 while "{0:.3f}".format(0.1234567890) would return 0.123.

In [0]:
"{:.3f}".format(0.1234567890)

'0.123'

In [0]:
list_solution = prob.solve()
print('Optimal value: {:.4f}'.format(list_solution))

Optimal value: 2.0000


We can also retrieve the values of $x$ and $y$.

In [0]:
print('x: {:.4f}, y: {:.4f}'.format(x.value, y.value))

x: 1.0000, y: 0.0000


### Matrix form

It is often convenient to express linear programs in a condensed, canonical form, and matrices are convenient for that. Rather than dealing with a set of scalar variables, we can express all variables in the problem as a vector $\mathbf{x}$. Likewise, we can store the coefficients of the constraints in a matrix $A$ and a vector $\mathbf{b}$ and the coefficients of the objective in a vector $\mathbf{c}$.


\begin{align*}
\text{minimize  } \mathbf{c}^T \mathbf{x} & \\
\text{subject to } A\mathbf{x} \leq \mathbf{b} \\
\mathbf{x} \geq 0
\end{align*}

Let's write the linear program above in this form. First, we'll rewrite the constraints to be consistent with the $A\mathbf{x} \leq \mathbf{b}$ constraint.

\begin{align*}
\text{minimize  }2x + 3y & \\
\text{subject to } -x - y &\leq -1 \\
-x + y &\leq 3 \\
2x+y &\leq 9 \\
x-y &\leq 3 \\
x &\geq 0 \\
y &\geq 0
\end{align*}

Let $\mathbf{x} = \begin{bmatrix} x & y \end{bmatrix}$. We can translate the linear program by reading off coefficients:

\begin{align*}
A = \begin{bmatrix}
-1 & -1 \\
-1 & 1 \\
2 & 1 \\
1 & -1 
\end{bmatrix} &&
\mathbf{b} = \begin{bmatrix} -1 \\ 3 \\ 9 \\ 3  \end{bmatrix}  &&
\mathbf{c} = \begin{bmatrix} 2 \\ 3 \end{bmatrix} 
\end{align*}

In [0]:
A = np.array([
    [-1, -1],
    [-1, 1],
    [2, 1],
    [1, -1]
])
b = np.array([-1, 3, 9, 3])
c = np.array([2, 3])

# We previously defined two scalar (one-dimensional) variables x and y.
# Here we define a two-dimensional vector x, which we call x_vec in our code
# to avoid confusion.
x_vec = cp.Variable(2)

In [0]:
matrix_prob = cp.Problem(cp.Minimize(cp.sum(c.T @ x_vec)),
                            constraints=[A@x_vec <= b, x_vec >= 0])

In [0]:
matrix_solution = matrix_prob.solve()

We can verify that the minimum value of the objective function is the same as above.

In [0]:
print('Optimal value (list form): {:.4f}'.format(two_var_solution))
print('Optimal value (matrix form):    {:.4f}'.format(matrix_solution))

Optimal value (list form): 2.0000
Optimal value (matrix form):    2.0000


Furthermore, the values of $\mathbf{x}$ correspond with the values of $x$ and $y$.

In [0]:
print('x (list) is {:.4f}  and x[0] (matrix) is {:.4f}'.format(x.value, x_vec.value[0]))
print('y (list) is {:.4f} and x[1] (matrix) is {:.4f}'.format(y.value, x_vec.value[1]))

x (list) is 1.0000  and x[0] (matrix) is 1.0000
y (list) is 0.0000 and x[1] (matrix) is -0.0000


## Problem 4: a diet problem (8 points)
Mackenzie, like a typical graduate student, wants to spend as little money as possible on food.  Somewhat less typically, she takes care to be sure she is still eating enough to hit her nutrition requriements:  calories (2,000 kcal), protein (55 g), and calcium (800 mg). Her local grocery store stocks six staple foods. The table below gives the nutrients and cost per serving of each food.

|Food|Energy (kcal)|Protein (g)|Calcium (mg)|Price (cents)|
|----|-------------|-----------|------------|-------------|
|Oatmeal|110|4|2|3|
|Chicken|205|32|12|24|
|Eggs|160|13|54|13|
|Milk|160|8|285|9|
|Cherry pie|420|4|22|20|
|Pork and beans|260|14|80|19|

Note that ten servings of pork and beans per day meets her nutritional requirements, but not even a grad student can eat like that! She decides to limit the number of servings per day she consumes for each food:

|Food|Max. servings|
|----|-------------|
|Oatmeal|4|
|Chicken|3|
|Eggs|2|
|Milk|8|
|Cherry pie|2|
|Pork and beans|2|

**Your task is to formulate and solve a linear program in the canonical form shown above to find the cheapest diet subject to these constraints.** The solution vector $\mathbf{x} \in \mathbb{R}^6$ should be ordered to correspond to this table—$\mathbf{x}_1$ servings of oatmeal, $\mathbf{x}_2$ servings of chicken, and so on. Report the values of $A$, $\mathbf{b}$, and $\mathbf{c}$ (you may find [the `\bmatrix` command](https://www.overleaf.com/learn/latex/Matrices) helpful in LaTeX when you're writing it up). Use CVXPY to solve the linear program; report the minimum cost (in cents) and the corresponding value of $\mathbf{x}$. Fractional servings are allowed.

## Problem 5: Exploring your diet solution (4 points)

Which constraints are extremized in the optimal solution?  

Relax one of those constraints by 1% and report by what percent that changed the optimum price.

Can Mackenzie find a feasible diet consisting of *only* eggs and cherry pie?  Equivalently, this asks whether the Eggs/CherryPie plane intersects the feasible polyhedron.

Report on any other observations that you can make by considering the geometry of the solution.